In [1]:
# ======================
# 📰 FAKE NEWS DETECTOR
# ======================

# 1️⃣ Installing required packages
!pip install streamlit pyngrok transformers torch pandas --quiet

# 2️⃣ Import libraries and configure ngrok
import os
from pyngrok import ngrok, conf

conf.get_default().auth_token = "33eEQATYeFwXNMBlPYBjYqGVq9y_5FTbXiK9V4YT8bUATN2RB"

# Kill previous tunnels if any
ngrok.kill()

# 3️⃣ Write Streamlit app to app.py
app_code = r'''
import streamlit as st
from transformers import pipeline
import torch
import pandas as pd

st.set_page_config(page_title="Fake News Detector", page_icon="📰", layout="centered")
st.title("📰 Fake News Detector for Students")
st.write("Analyze any news article or social media post to detect reliability and get a short summary.")

# Sidebar options
st.sidebar.header("⚙️ Options")
max_len = st.sidebar.slider("Max summary length", 30, 150, 60)
min_len = st.sidebar.slider("Min summary length", 10, 80, 25)
show_summary = st.sidebar.checkbox("Show summary", True)
st.sidebar.markdown("---")
st.sidebar.caption("Developed by **Roshan J** | © 2025")

# Load models
@st.cache_resource
def load_models():
    device = 0 if torch.cuda.is_available() else -1
    classifier = pipeline(
        "text-classification",
        model="jy46604790/Fake-News-Bert-Detect",
        tokenizer="jy46604790/Fake-News-Bert-Detect",
        device=device,
    )
    summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=device)
    return classifier, summarizer

classifier, summarizer = load_models()

# Initialize history
if "history" not in st.session_state:
    st.session_state.history = []

# User input
text_input = st.text_area("🗞 Paste your news article or post here:", height=250)

# Analyze button
if st.button("🔍 Analyze"):
    if not text_input.strip():
        st.warning("⚠️ Please enter some text first.")
    else:
        with st.spinner("Analyzing the text..."):
            result = classifier(text_input)[0]
            label = result["label"]
            score = round(result["score"] * 100, 2)
            verdict = "FAKE" if label=="LABEL_0" else "REAL"

            summary = summarizer(text_input, max_length=max_len, min_length=min_len, do_sample=False)[0]["summary_text"]

        st.subheader("📊 Credibility Analysis")
        col1, col2 = st.columns(2)
        col1.metric("Verdict", verdict)
        col2.metric("Confidence", f"{score}%")

        if verdict.lower() == "fake":
            st.error(f"🚫 FAKE / Unreliable — Confidence: {score}%")
        else:
            st.success(f"✅ REAL / Credible — Confidence: {score}%")

        if show_summary:
            st.subheader("📝 Summary")
            st.info(summary)

        query = "+".join(text_input[:60].split())
        st.markdown(f"[🔎 Verify on Google News](https://news.google.com/search?q={query})")

        st.session_state.history.append({
            "text": text_input[:150]+"..." if len(text_input)>150 else text_input,
            "label": verdict,
            "score": f"{score}%",
            "summary": summary
        })

# History
if st.session_state.history:
    st.markdown("---")
    st.subheader("📜 Analysis History")
    df = pd.DataFrame(st.session_state.history)
    st.download_button("⬇️ Download History as CSV", data=df.to_csv(index=False), file_name="history.csv", mime="text/csv")
    if st.button("🗑️ Clear History"):
        st.session_state.history.clear()
        st.success("History cleared!")
    for i, record in enumerate(reversed(st.session_state.history), 1):
        with st.expander(f"📰 Entry #{i} — {record['label']} ({record['score']})"):
            st.write("**🗞 Original (truncated):**", record["text"])
            st.write("**📝 Summary:**", record["summary"])

st.markdown("---")
st.caption("Made with ❤️ using Streamlit & Transformers | Developed by Roshan J")
'''

# app code to file
with open("app.py", "w") as f:
    f.write(app_code)

# 4️⃣ Run Streamlit in the background
os.system("nohup streamlit run app.py --server.port 8501 > /dev/null 2>&1 &")

# 5️⃣ Start ngrok tunnel
public_url = ngrok.connect(8501)
print(f"🚀 Your public URL: {public_url}")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 73.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 93.3 MB/s eta 0:00:00
🚀 Your public URL: NgrokTunnel: "https://objurgative-sensationistic-georgiann.ngrok-free.dev" -> "http://localhost:8501"
